In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)


___
### Tags: #Source #Asgard

### Links: 
___
# creating a table from s3

In [20]:
query_text = """--sql 
   CREATE TABLE db1.cameras_dir_asgard
(
    `camera_serial` String,
    `intercom_uuid` String,
    `camera_uuid` String,
)
ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/cameras_dir_asgard/*.csv','CSVWithNames')
    """

In [25]:

query_text = """--sql
    CREATE TABLE db1.cameras_dir_asgard_ch
(
    `camera_serial` String,
    `intercom_uuid` String,
    `camera_uuid` String,
)
    ENGINE = MergeTree()
    ORDER BY intercom_uuid
    """
ch.query_run(query_text)

""


In [27]:

query_text = """--sql
    CREATE MATERIALIZED VIEW db1.cameras_dir_asgard_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.cameras_dir_asgard_ch AS
    SELECT
        *
    FROM db1.cameras_dir_asgard
    """

ch.query_run(query_text)

""


In [28]:
query_text = """--sql
    SELECT
    *
    FROM db1.cameras_dir_asgard_ch
    LIMIT 10
    """

ch.query_run(query_text)

,camera_serial,intercom_uuid,camera_uuid
0,EF000000006DCA21,,5d4c3ea7-f360-43d5-b00a-758b5d462dab
1,DC13290200018,,c0fa6553-489c-40b5-96e0-9801838e16f4
2,EF0000000522E686,,e6439257-aa0b-46bc-8dc4-40eac8054209
3,EF0000000104C520,,1bc4771b-89ee-43fd-a39b-419c1f47a00a
4,EF00000006113D1A,,e2899c75-e333-4bf6-9003-6141ed4d2ed7
5,EF00000006113F2D,,59bf862f-60aa-4633-a0ff-15be0ae76f2a
6,EF00000006113C69,,a630b17d-cb14-40c8-a733-04db621d6ae7
7,EF000000007F2DA4,,2b918989-5e58-454b-a89e-0e9abe769c43
8,EF00000005053673,,1a3fb751-d11e-4a24-ab7c-a10284ba7b3a
9,EF000000050535D6,,04af64e5-14f2-490c-840f-834680cd9ea5


In [9]:
query_text = """--sql
    TRUNCATE TABLE db1.cameras_dir_asgard_ch
    """

ch.query_run(query_text)


""


In [22]:
query_text = """--sql
    DROP TABLE db1.cameras_dir_asgard_ch
    """

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 60
 Code: 60. DB::Exception: Table db1.cameras_dir_asgard_ch does not exist. Maybe you meant db1.cameras_dir_asgard_ch?. (UNKNOWN_TABLE) (version 25.3.6.56 (official build))
